In [1]:
yaml_string = """yaml
- route:
    id: my-basic-route
    from:
      uri: direct:start
    steps:
      - log:
          message: "Processing message: ${body}"
"""

In [2]:
prompt = f"""You are an camel integration expert that make changes to the yaml file and return me the new yaml file

Here is my original YAML files that I want you to make change:
{yaml_string}

Can you add a transformation step to this YAML file?

After making the change, just repond me the new YAML file in yaml file format.
"""

In [5]:
import requests

def query_ollama(prompt, model="codellama:7b"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False,
            "temperature": 0.2,
            "top_p": 0.7,
            "num_predict": 500,
        }
    )
    return response.json()["response"]

#prompt = "Generate a YAML file for a basic GitHub Actions workflow that runs tests on Python 3.10"
prompt = prompt.strip()
print(query_ollama(prompt))


Sure! Here is the updated YAML file with the added transformation step:
```yaml
- route:
    id: my-basic-route
    from:
      uri: direct:start
    steps:
      - log:
          message: "Processing message: ${body}"
      - transform:
          type: xpath
          expression: /Order/@id
```
In this updated YAML file, we have added a new step to the route named `transform`. This step uses an XPath expression to extract the value of the `@id` attribute from the `Order` element. The resulting value is then used as the input for the next step in the route, which in this case is the log step.

Note that you may need to modify the XPath expression depending on the structure of your message data.


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# Load the YAML file as a document
loader = TextLoader("./data/DSL_example.txt", encoding="utf-8")
documents = loader.load()

# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

# Initialize the Ollama embeddings model
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Create a FAISS vector store from the text chunks
vector_store = FAISS.from_documents(texts, embeddings)

retriever = vector_store.as_retriever()

# Define the YAML string and instructions
instructions = '''add a transformation to this YAML file'''

yaml_string = """
- route:
    id: my-basic-route
    from:
      uri: direct:start
    steps:
      - log:
          message: "Processing message: ${body}"
"""

# Define a prompt template for the LLM chain
prompt_template = PromptTemplate.from_template('''
you are an expert in YAML files and apache camel integration.
Context:
{context}
Here is a YAML file that I want you to make change:
{yaml_string}
Can you make changes to this YAML file based on the following instructions?
{instructions}
After making the change, just respond with the new YAML file in YAML format.
''')

# Initialize the LLM chain with the Ollama model
llm = Ollama(model="codellama:7b")  

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
) 

# Run the chain with the YAML string and instructions
result = chain.invoke({
    "yaml_string": yaml_string,
    "instructions": instructions
})

# Print the result
print(result['text'])










ValidationError: 1 validation error for LLMChain
retriever
  Extra inputs are not permitted [type=extra_forbidden, input_value=VectorStoreRetriever(tags...C680>, search_kwargs={}), input_type=VectorStoreRetriever]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [8]:
import os
import base64
print(base64.b64encode(os.urandom(32)).decode('utf-8'))

6ztZcFP+kx60DZGtQioeTWPygk41szqhSvdeKPLorjo=


In [7]:
import os
import binascii
print(binascii.hexlify(os.urandom(32)).decode('utf-8'))

64334057e270c32729e5d265aa9297fb5c48a95035d1f40ab11cee6345c72253
